                                                                      Marcus Eriksson, Mattias Barth, Sara Kaminska

# TF-IDF - vad är det och hur fungerar det?
TF-IDF står för Term Frequency - Inverse Document Frequency och är en teknik som används för att statistiskt avgöra hur relevant ett ord är för ett dokument i en dokumentsamling. Tekniken används vid automatiserad textanalys i bland annat sökmotorer.

Term Frequency är den delen av tekniken som beräknar hur vanligt förekommande ett ord är i ett dokument. I beräkningen tas det hänsyn till dokumentets omfattning, genom att dividera antalet gånger ordet förekommer med det totala antalet ord. 

    TF = antalet gånger ordet förekommer i dokumentet / totala antalet ord i dokumentet

Inverse Document Frequency har som uppgift att värdera hur relevant ett ord är för ett dokument. En text innehåller många så kallade stop-ord, exempelvis I, we, it, did, here, vilket leder till att de får ett högt TF-värde. Att ta hänsyn till dessa stop-ord i analysen ger emellertid ett vilseledande resultat. Det är här IDF kommer in i bilden. Tekniken kontrollerar hur många dokument i en samling innehåller det givna ordet. Vidare divideras det totala antalet dokument i samlingen med antalet dokument som ordet förekommer i. Eftersom värdena kan bli väldigt höga för större dokumentsamlingar används logaritmen av 10, för att minska de. Beräkningen resulterar i att ord som förekommer i färre dokument får en högre siffra, och därmed ett högre värde. 

	IDF = antalet dokument i samlingen / antalet dokument som innehåller det givna ordet

Det slutliga värdet för TF-IDF räknas ut genom att multiplicera TF (antalet gånger ett ord förekommer i en text)  med IDF (ordets vikt). Ett ord som förekommer ofta i många dokument kommer att få ett lägre TF-IDF, medan ett ord som förekommer ofta i något enstaka dokument får ett högre, vilket tolkas som att ordet har en hög relevans för dokumentet. 

	TF-IDF = TF・IDF

# Hur fungerar TF-IDF i praktiken?

Det första som görs är att analysera det "dataset" man har för att veta hur datan skall kunna jämföras, analyseras och tas fram. Den data som skall analyseras är inte alltid trivial att ta fram och därför bör det åtminstone göras en mindre analys av datan (som kan göras manuellt). När den första analysen är klar och man har hittat ett sätt att få fram sin data behöver man processa den aktuella datan. Datan behöver tvättas och omvandlas då maskininlärningsalgoritmer inte direkt kan arbeta med råtext. Texten behöver konverteras till vektorer som algoritmen klarar av att använda. De metoder som används för att tvätta data varier från fall till fall. Några exempel på metoder för datatvätt av en text är:

- gör all text till "lover case"
- ersätta skiljetecken (" . , # _) i texten till blanksteg
- ersätta ord med bara ett tecken till blanksteg
- ta bort "stop words" från texten. "Stop words" är de mest vanliga orden i språket och ger inget användbart värde. Vidare öka effektiviteten i beräkningar och i utrymme om dessa “stop words” tas bort.
- "stemming", där längre ord omvandlas till sin stam
- "lemmatisation, där ord reduceras till "root synonym"

När datatvätten är klar kan beräkningar och analyser göras. 


# För- och nackdelar med TF-IDF
Nedan analys bygger på våra erfarenheter av TF-IDF. De fördelar som vi upplever med tekniken är att det är ett bra och grundläggande verktyg med en enkel algoritm som ger en god generell bild av vikten av sökta ord/dokument mot ett "corpus". TF-IDF verkar vara en bra start för att börja förstå grunderna inom Machine Learning. 


Några nackdelar som vi upptäckt under tiden som vi arbetat med TF-IDF är att tekniken kräver ett stort corpus för att ge bra output. Ett stort corpus medför att det tar tid att göra beräkningarna då det är många dokument att jämföra med.Ytterligare en nackdel är att man måste veta med vilka metoder man skall tvätta sin data, dels för att för att kunna få fram relevanta resultat, dels att det riskerar att bli tungt att köra annars. Vi upplevde även att det var svårt att tolka resultatet och veta exakt vad de siffervärden vi får fram innebär. 


# Bokrekommendation 
Om vi hade haft mer tidsutrymme hade vi angripit uppgiften om bokrekommendation enligt nedan. Vi hade börjat med att skapa en dokumentsamling med böcker som vi kan göra våra rekomendationer utifrån. Vidare hade vi gjort en TF-IDF analys på samtliga och lagrat resultaten. När en användare sedan ber om en bokrekommendation vektoriseras samtliga dokument som finns sparade och de med högst cosine similarity, det vill säga högst likhet, rekomenderas till användaren. Ytterligare ett villkor som förbättrar bokrekommendationerna och effektiviserar sökningen är att även lagra information om genre för varje bok. Vår algoritm skulle isåfall först begränsa sökningen till böckerna i samma genre, och endast beräkna cosine similarity för dessa. 


# Källförteckning
Toward Data Science, TF-IDF from scratch in python on real world dataset, hämtad 2021-03-11 från
https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-pythonon-real-world-dataset-796d339a4089

Towards Data Science, TF-IDF Python Example, hämtad 2021-03-14 från https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76


In [2]:
import math
import os
import pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Koden nedan ber användaren om en boktitel att söka på. Sedan görs sökning mot websidan gutenberg.org och användaren presenteras de 25 mest populära böckerna som matchar sökningen. Boken tvättas (detta förklaras djupare längre ner) och sparas ner i en fil med hjälp av biblioteket pickle. Processen upprepas en gång till för att få en titel att jämföra med.


In [2]:
def save_books():
    for i in range(2):
        search = input("Search: ")
        book_lines = scrape(search, i)
        cleaned = clean_book(book_lines)
        with open(f'documents/book{i+1}.py', 'wb') as file:
            pickle.dump(cleaned, file)


def scrape(title, j):
    driver = webdriver.Chrome('./chromedriver')
    driver.get('https://www.gutenberg.org/')
    search_field = driver.find_element_by_id('menu-book-search')
    search_field.send_keys(title.lower())
    search_field.send_keys(Keys.RETURN)
    results = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'booklink')))
    titles = [result.find_element_by_class_name("title").text for result in results]

    for i, book in enumerate(titles):
        print(f"{i + 1}. " + book)

    choice = int(input("What book were you thinking of?(1-25): "))

    chosen_book = results[choice+1]
    chosen_book.click()
    text = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.LINK_TEXT, 'Plain Text UTF-8')))
    text.click()
    book_text = driver.page_source
    with open(f"documents/book{j+1}.py", "w") as text_file:
        text_file.write(book_text)
    with open(f"documents/book{j+1}.py", "r") as text_file:
        book_lines = text_file.readlines()
    return book_lines

Här tvättas boktexterna. Följande processer går texten igenom: 
    - Plocka ut det faktiska boktexten. Vi tar bort all text som inte hör boken till, exempelvis policy.
    - Tar bort radbrytningar. Eftersom boken ligger i en textfil (ex. bok.txt) ligger radbrytningarna('\n') som en del       av texten när raderna läses in som enskilda element i en lista. Dessa vill vi bli av med. 
    - Listan med rader görs om till en sträng med all text. 
    - Samtliga tecken som inte är bokstäver tas bort från texten. Apostrofer tas även bort. Don't ---> Dont
    - Texten delas upp ord för ord och sparas i en lista.
    - Eventuella element i listan som bara består av mellanslag tas bort. Mellanslag i början och/eller slutet av           orden tas även bort. 
    - Enbokstavsord tas bort från listan, då detta inte bidrar med någon semantisk vikt till texten
    - Stopord tas bort från texten då dessa heller inte ger någon semantisk vikt till texten. Ord såsom The, Them, I,       Me, My etc.

In [ ]:
def remove_unwanted_text(lines):
    for i, line in enumerate(lines):
        if "*** START OF" in line or "***START OF" in line:
            start = i + 1
        if "*** END OF" in line or "***END OF" in line:
            end = len(lines) - i
    book = lines[start:-end:]
    return book


def remove_line_br(book):
    book = [line for line in book if line != '\n']
    for i, line in enumerate(book):
        if "\n" in line:
            book[i] = line.rstrip("\n")
    return book


def to_string(book):
    book_str = ""
    for line in book:
        book_str += line + " "
    return book_str


def no_symbols(book_string):
    alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    unsymboled_string = ""
    for character in book_string:
        if character == "'":
            unsymboled_string += ""
        elif character not in alphabet:
            unsymboled_string += " "
        else:
            unsymboled_string += character.lower()
    return unsymboled_string


def to_word_list(book_string):
    return book_string.split(" ")


def no_blanks(word_list):
    no_blanks_word_list = []
    for word in word_list:
        if word != " ":
            no_blanks_word_list.append(word)
    for i, line in enumerate(no_blanks_word_list):
        if " " in line:
            no_blanks_word_list[i] = line.rstrip(" ")
    return no_blanks_word_list


def remove_one_letter_words(words_list):
    no_one_letter_words_list = [word for word in words_list if len(word) > 1]
    return no_one_letter_words_list


def remove_stopwords(word_list):
    stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'youre', 'youve', 'youll', 'youd', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'shes', 'her', 'hers', 'herself', 'it', 'its', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thatll', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'dont', 'should', 'shouldve', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'arent', 'couldn', 'couldnt', 'didn', 'didnt', 'doesn', 'doesnt', 'hadn', 'hadnt', 'hasn', 'hasnt', 'haven', 'havent', 'isn', 'isnt', 'ma', 'mightn', 'mightnt', 'mustn', 'mustnt', 'needn', 'neednt', 'shan', 'shant', 'shouldn', 'shouldnt', 'wasn', 'wasnt', 'weren', 'werent', 'won', 'wont', 'wouldn', 'wouldnt']
    no_stopwords_word_list = [word for word in word_list if word not in stopwords]
    return no_stopwords_word_list

def clean_book(book_lines):
    data = remove_unwanted_text(book_lines)
    data = remove_line_br(data)
    data = to_string(data)
    data = no_symbols(data)
    data = to_word_list(data)
    data = no_blanks(data)
    data = remove_one_letter_words(data)
    cleaned_data = remove_stopwords(data)
    return cleaned_data

Funktionerna laddar in filen och sparar den i en lista för vidare hantering. 

In [ ]:
def get_documents():
    with open("documents/book1.py", "rb") as book1:
        book1 = pickle.load(book1)
    with open("documents/book2.py", "rb") as book2:
        book2 = pickle.load(book2)
    return book1, book2

def get_corpus():
    corpus_list = []
    for file in os.listdir('./corpus'):
        with open('./corpus/' + file, 'rb') as book_file:
            book_file = pickle.load(book_file)
            corpus_list.append(book_file)
    return corpus_list


Funktionerna ansvarar för att utföra samtliga beräkningar som krävs för att få fram ett TF-IDF värde. 

In [ ]:
def calculate_tf(book):
    tf = {}
    for word in book:
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1
    for key in tf:
        tf[key] = tf[key]/len(book)
    return tf


def calculate_df(book, corpus):
    df = {}
    for i, word in enumerate(book):
        print(f"{i+1}/{len(book)}")
        df[word] = 0
        for corpus_document in corpus:
            if word in corpus_document:
                df[word] += 1
    return df


def calculate_idf(df):
    n = len(os.listdir('./corpus'))
    for key in df:
        df[key] = math.log(n/(df[key]+1), 10)
    idf = df
    return idf


def calculate_tf_idf(tf, idf):
    tf_idf = {}
    for key in tf:
        tf_idf[key] = tf[key] * idf[key]
    return tf_idf

Slutligen analyseras TF-IDF värdet genom att vektorisera böckerna och sedan beräkna vinkeln emellan dem. Detta görs med hjälp av följande förhållande: a ∙ b = |a||b| cos(v) där a och b är vektorer och v är vinkeln emellan dem.

In [ ]:
def get_total_vocab(book1, book2, corpus):
    total_vocab = []
    for corpus_document in corpus:
        total_vocab += corpus_document
    total_vocab += book1 + book2
    return list(set(total_vocab))


def vectorize_book(current_book, total_vocab):
    book_vector = []
    for word in total_vocab:
        if word in current_book:
            current_tfidf = current_book[word]
            value = current_tfidf
        else:
            value = 0

        book_vector.append(value)

    return book_vector


def calculate_cosine_similarity(a, b):
    print("Len a = ", len(a))
    print("Len b = ", len(b))

    dot_product = sum([a[i]*b[i] for i in range(len(a))])
    absolute = math.sqrt(sum([i**2 for i in a]) * sum([i**2 for i in b]))

    return math.degrees(math.acos(dot_product/absolute))

Nedan körs alla nödvändiga funktioner och returnerar en uppskattning av böckernas likhetsgrad.

In [ ]:
def compare_books(book1, book2, corpus):
    tf1 = calculate_tf(book1)
    df1 = calculate_df(book1, corpus)
    idf1 = calculate_idf(df1)
    tf_idf1 = calculate_tf_idf(tf1, idf1)

    tf2 = calculate_tf(book2)
    df2 = calculate_df(book2, corpus)
    idf2 = calculate_idf(df2)
    tf_idf2 = calculate_tf_idf(tf2, idf2)

    total_vocab = get_total_vocab(book1, book2, corpus)

    book1_vector = vectorize_book(tf_idf1, total_vocab)
    book2_vector = vectorize_book(tf_idf2, total_vocab)
    cosine_similarity = get_cosine_similarity(book1_vector, book2_vector)

    appreciation = None

    if cosine_similarity == 0:
        appreciation = "Böckerna är likadana"
    elif 0 < cosine_similarity <= 10:
        appreciation = "Böckerna har väldigt många likheter"
    elif 10 < cosine_similarity <= 45:
        appreciation = "Böckerna har många likheter"
    elif 45 < cosine_similarity <= 60:
        appreciation = "Böckerna har vissa likheter"
    elif 60 < cosine_similarity <= 80:
        appreciation = "Böckerna har få likheter"
    elif 80 < cosine_similarity:
        appreciation = "Böckerna har väldigt få likheter"
    return appreciation, cosine_similarity

In [4]:
save_books()
book1, book2 = get_documents()
corpus = get_corpus()
appreciation, cosine_similarity = compare_books(book1, book2, corpus)
print("Appreciation: ", appreciation)
print("Cosine similarity: ", cosine_similarity)

Search: TWILIGHT


NameError: name 'webdriver' is not defined